In [ ]:
from functools import partial

def l2_reg(coef=1e-2):
    #tf.keras.regularizers.l2()를 재구현
    return lambda x:tf.reduce_sum(x**2) * coef

class ConvWithRegularizers(SimpleConvolutionLayer):
    def __init__(self,num_kernels=32,kernel_size=(3,3),stride=1,
                 kernel_regularizer=l2_reg(),bias_regularizer=None):
        super().__init__(num_kernels,kernel_size,stride)
        self.kernel_regularizer=kernel_regularizer
        self.bias_regularizer = bias_regularizer
        
    def build(self,input_shape):
        super().build(input_shape)
        #정규화손실을 변수에 추가
        if self.kernel_regularizer is not None:
            #예를 들어 TF에 호출(반복)마다
            #tf.nn.l1_loss(self.kernels)를 계산하고 저장할 것을 지시
            self.add_loss(partial(self.kernel_regularizer,self.kernels))
        
        if self.bias_regularizer is not None:
            self.add_loss(partial(self.bias_regularizer,self.bias))
            
    
        

In [ ]:
model = Sequential()

model.add(ConvWithRegularizers(6,(5,5),kernel_regularizer=l2_reg()))
model.add(...)
model.add(Dense(num_classes,activation='softmax'))

for epoch in range(epochs):
    for (batch_images,batch_gts) in dataset:
        with tf.GradientTape() as grad_tape:
            loss = tf.losses.sparse_categorical_crossentropy(
                batch_gts,model(batch_images)) #주요 손실
            loss += sum(model.losses) #추가 손실 리스트
        
        #결합된 손실의 경사를 구해 역전파
        grads = grad_tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_varibales))
            
        

In [ ]:
l1_reg = tf.keras.regularizers.l1(0.01)

model = Sequential()
model.add(Conv2D(6,kernel_size=(5,5),padding='same',activation='relu',input_shape=input_shape,kernel_regularizer=l1_reg))